In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("https://raw.githubusercontent.com/mwitiderrick/kerasDO/master/HR_comma_sep.csv")

In [2]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [3]:
feats = ['department','salary']
df_final = pd.get_dummies(df,columns=feats,drop_first=True)

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X = df_final.drop(['left'],axis=1).values
y = df_final['left'].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense

/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
classifier = Sequential()

In [10]:
classifier.add(Dense(9, kernel_initializer = "uniform",activation = "relu", input_dim=18))

In [11]:
classifier.add(Dense(1, kernel_initializer = "uniform",activation = "sigmoid"))

In [12]:
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

In [13]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 1)

Epoch 1/1
10499/10499 [==============================] - 2s 205us/step - loss: 0.4244 - acc: 0.7969


In [14]:
y_pred = classifier.predict(X_test)

In [15]:
y_pred = (y_pred > 0.5)

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[3300,  111],
       [ 624,  465]])

In [17]:
new_pred = classifier.predict(sc.transform(np.array([[0.26,0.7 ,3., 238., 6., 0.,0.,0.,0., 0.,0.,0.,0.,0.,1.,0., 0.,1.]])))

In [18]:
new_pred = (new_pred > 0.5)
new_pred

array([[False]])

In [19]:
new_pred = (new_pred > 0.6)
new_pred

array([[False]])

In [20]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [21]:
def make_classifier():
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [22]:
classifier = KerasClassifier(build_fn = make_classifier, batch_size=10, nb_epoch=1)

In [23]:
accuracies = cross_val_score(estimator = classifier,X = X_train,y = y_train,cv = 10,n_jobs = -1)

Epoch 1/1
1050/1050 [==============================] - 0s 140us/step
Epoch 1/1
1050/1050 [==============================] - 0s 149us/step
Epoch 1/1
1050/1050 [==============================] - 0s 174us/step
Epoch 1/1
1050/1050 [==============================] - 0s 166us/step
Epoch 1/1
1050/1050 [==============================] - 0s 177us/step
Epoch 1/1
1050/1050 [==============================] - 0s 180us/step
Epoch 1/1
1050/1050 [==============================] - 0s 200us/step
Epoch 1/1
1050/1050 [==============================] - 0s 210us/step
Epoch 1/1
1050/1050 [==============================] - 0s 225us/step
Epoch 1/1
1049/1049 [==============================] - 0s 233us/step


In [24]:
mean = accuracies.mean()
mean

0.8290297287359474

In [25]:
variance = accuracies.var()
variance

0.00045488474371275123

In [26]:
from keras.layers import Dropout

classifier = Sequential()
classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
classifier.add(Dropout(rate = 0.1))
classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

In [27]:
from sklearn.model_selection import GridSearchCV
def make_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= optimizer,loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [28]:
classifier = KerasClassifier(build_fn = make_classifier)

In [29]:
params = {
    'batch_size':[20,35],
    'epochs':[2,3],
    'optimizer':['adam','rmsprop']
}

In [30]:
grid_search = GridSearchCV(estimator=classifier,
                           param_grid=params,
                           scoring="accuracy",
                           cv=2)

In [31]:
grid_search = grid_search.fit(X_train,y_train)

Epoch 1/2
5249/5249 [==============================] - 1s 228us/step - loss: 0.5958 - acc: 0.7645
Epoch 2/2
5249/5249 [==============================] - 0s 82us/step - loss: 0.3962 - acc: 0.8510
Epoch 1/2
5250/5250 [==============================] - 1s 222us/step - loss: 0.5935 - acc: 0.7596
Epoch 2/2
5250/5250 [==============================] - 0s 85us/step - loss: 0.4080 - acc: 0.8029
Epoch 1/2
5249/5249 [==============================] - 1s 214us/step - loss: 0.5929 - acc: 0.7676
Epoch 2/2
5249/5249 [==============================] - 0s 82us/step - loss: 0.4261 - acc: 0.7864
Epoch 1/2
5250/5250 [==============================] - 1s 221us/step - loss: 0.6011 - acc: 0.7552
Epoch 2/2
5250/5250 [==============================] - 0s 84us/step - loss: 0.4439 - acc: 0.7869
Epoch 1/3
5249/5249 [==============================] - 1s 248us/step - loss: 0.5645 - acc: 0.7687
Epoch 2/3
5249/5249 [==============================] - 0s 86us/step - loss: 0.4090 - acc: 0.7702
Epoch 3/3
5249/5249 [====

In [32]:
best_param = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [33]:
best_param

{'batch_size': 20, 'epochs': 2, 'optimizer': 'adam'}

In [34]:
best_accuracy

0.8533193637489285